# 2.9 大模型应用生产实践

## 🚄 前言

通过前面课程学习，你的答疑机器人已经可以比较好的回答私域知识，接下来将继续探讨如何将这个大模型应用发布至生产环境，即如何将大模型应用从开发和测试阶段转移到实际业务场景中，这是一个复杂且关键的过程，涉及多个步骤和技术考虑。本节课程将讨论它们帮助你理解并顺利完成这一过程。

## 🍁 课程目标

学完本课程后，你将能够：
* 通过业务需求分析明确将大模型应用发布至生产环境的关键要素
* 了解如何平衡大模型应用的性能和运行成本
* 了解如何提升大模型应用的稳定性
* 了解如何保障大模型应用安全合规

将大模型发布至生产环境并应用于实际业务场景并非易事。这不仅需要从业务场景出发，通过明确的需求选择更适合的模型（模型的功能性需求，如针对数学问题优化的Qwen-Math）。还需要从系统架构的角度全面考量性能、成本、安全性和稳定性等非功能性需求，模型的非功能性需求虽然不直接关联到具体功能，但对模型的整体质量和用户体验至关重要。

总体来说，功能性需求定义了大模型“做什么”，而非功能性需求则确保其在执行这些功能时“如何做”才能达到高质量标准，只有在业务需求和技术实现之间找到平衡，才能高效部署和运营大模型服务。

本节课程将围绕这些核心主题展开，帮助你全面了解如何将大模型高效、低成本地部署到实际业务场景中，并为如何搭建稳定、安全的系统架构提供指导。

## 1. 业务需求分析
业务需求分析是成功部署大模型的第一步。不同的业务场景对模型的功能性需求和非功能性需求差异巨大。如果业务场景不清晰，可能会导致以下问题：
* 模型选择错误：选择了不适合特定任务的模型，导致性能不佳或资源浪费。
* 用户体验下降：无法满足用户对实时性、准确性等关键指标的需求。
* 成本失控：未根据业务需求优化模型规模和部署方案，导致高昂的计算和运维成本。
* 安全隐患：未能充分考虑数据隐私和合规性要求，可能引发法律风险。

因此，明确业务场景后，你需要进一步围绕功能性需求和非功能性需求进行深入分析，并据此制定具体的部署方案。

### 1.1 模型的功能性需求
不同业务场景对模型的要求各不相同，以下是一些典型任务场景下的模型选择推荐：
* 自然语言处理：这是最常见的大模型应用场景之一，主要包括问答系统、文本生成、翻译、情感分析等任务。
    * 通用任务：如开放域问答、新闻摘要生成等，可以使用通用的大语言模型（如Qwen、GPT等）。
    * 特定领域任务：对于数学解题、法务咨询、医疗诊断等专业领域，建议选择经过领域微调的模型。例如：
        * 数学问题：可以选择专门针对数学任务优化的模型，如Qwen-Math。
        * 法务问题：可以选择面向法律领域训练的模型，如通义法睿。
        * 医疗诊断：需要高准确性和专业术语支持的模型，通常需要结合领域知识图谱或规则引擎。
* 视觉：包括图像分类、目标检测、图像生成等。这类任务通常需要专门的视觉模型（如通义万相、YOLO、Stable Diffusion等），而不是通用的语言模型。
* 语音：如语音助手、自动字幕生成、语音输入法、语音合成等。这类任务通常需要专门的语音处理模型（如Qwen-Audio、CosyVoice等）
* 多模态任务：结合文本、图像、视频、语音等多种模态，处理复杂任务。这里建议你使用专门设计的多模态模型（如Qwen-VL）能够显著提升效率和一致性，因为简单组合多个单模态模型（如语音识别 + 文本生成 + 图像理解）来完成多模态任务，会导致应用整体的延迟较高、一致性差、开发复杂度高等问题。

确定任务场景后，你可能还会发现有很多功能类似的大模型可供选择（如Qwen、DeepSeek、GPT等），推理的**准确度**可能是你选择模型的最主要原因，你需要**自己构建评测数据集**或**选择与业务场景匹配的公开数据集**进行评测（如MMLU评估语言理解、BBH测试复杂推理等）。
<img src="https://img.alicdn.com/imgextra/i2/O1CN01YFnJL820aE1wiLRgS_!!6000000006865-0-tps-2832-1118.jpg" width="900">

图片来源：[开源大模型评测排行](https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard#/) （需要外网访问权限）

### 1.2 模型的非功能性需求

在正式地将模型应用于生产之前，除了选择更适合于业务场景的模型外（功能性需求），通常还需要关注**性能、成本、稳定、安全**等方面（非功能性需求）。在本小节接下来的内容中，将分块展开讨论这些内容。

<br>



## 2. 性能优化

你的业务是否在意响应速度？例如，对话系统通常要求快速反馈（通常低于500ms），而离线批处理任务对延迟要求较低（最长可以是数小时甚至数天）。通常来说你需要明确**服务化级别目标 SLO（Service Level Object）**，这是云厂商提供LLM部署服务时给客户保证的一个性能指标，常见的指标有 **TTFT（Time to first token latency，首 token 延时）** 和 **TPOT（Time per output token，每 token 生成时间）**。不同的业务场景的可选性能评估数据集和 SLO 要求参考如下：

| 业务场景            | 常用性能评估数据集  | TTFT 要求  | TPOT 要求  |
|-------------------|------------------|------------|-----------|
| 对话、咨询、搜索类         | ShareGPT，MMLU | 高 | 中 |
| 代码补全、编程、网页设计    | HumanEval      | 高 | 高 |
| 阅读理解/总结/数据处理/信息提取 | LongBench | 低 | 中 |
| 通用大模型（DeepSeek R1，通义大模型等） | InfoVQA 等多模态评估数据集 | TTFT < 5sec（推荐小于该值） | TPOT < 200ms（推荐小于该值）|

当确定了系统所需要的 SLO 服务级别目标（包括TTFT要求和TPOT要求）后，就可以围绕这些目标对大模型应用进一步优化。

### 2.1 系统性能提升

下面列举了一些提升 LLM 系统性能的原则，这些原则是通用的，与业务无关，良好的应用这些原则可以帮助你从多个角度降低系统延迟，提升用户体验。

#### 2.1.1 更快地处理请求

通常影响模型推理速度的最主要因素是**模型的大小**，较小规模的模型可以更快地完成推理。

对于简单业务场景，你可以直接选择参数较小的模型（如Qwen2-7b），也可以通过模型压缩与量化的方式加速推理速度，常见的方法有：
* 模型剪枝：移除模型中冗余的权重或层，降低模型复杂度。
* 量化：使用 INT4、INT8 或 FP16 量化技术，减少模型推理所需的计算资源。
* 知识蒸馏：使用知识蒸馏技术训练小型模型，替代大型模型完成推理任务。

| 模型剪枝 | 量化 | 知识蒸馏 |
|--------|-----|---------|
| <img src="https://img.alicdn.com/imgextra/i2/O1CN01qWEBtx1gif0bta4YW_!!6000000004176-0-tps-1448-784.jpg" width="308"> | <img src="https://img.alicdn.com/imgextra/i1/O1CN01C4UG8o1Y18RZPZXFJ_!!6000000002998-2-tps-680-486.png" width="234"> | <img src="https://img.alicdn.com/imgextra/i2/O1CN018yxBtQ1FtCUwbMrY6_!!6000000000544-0-tps-1969-807.jpg" width="310"> <br> 图片来源：[Knowledge Distillation: A Survey](https://arxiv.org/pdf/2006.05525)|

在前面章节中，讨论过如何让较小的模型也能够高质量地完成推理，包括：
* 优化提示词：你可以通过提示词扩写来提供更详细的提示，也可以添加更多的样本来指引模型更好地推理。
* 微调模型：在特定领域/任务中，小模型微调后可能接近甚至超越大模型。

<br>

#### 2.1.2 减少大模型处理请求数和运算量

通过减少大模型处理的请求数量或运算量，可以降低硬件负载（如GPU）和并发压力，缩短排队和推理时间，从而降低系统整体延迟，提升性能。
* 上下文缓存：使用文本生成模型时，不同的推理请求之间可能会有重合的输入内容（如多轮对话、针对一本书的多次提问等），上下文缓存（Context Cache）技术可以将这些请求的公共前缀内容进行缓存，在推理时减少重复的运算量，提升响应速度。通义千问系列模型（qwen-max、 qwen-plus 、qwen-turbo）默认支持[上下文缓存（Context Cache）](https://help.aliyun.com/zh/model-studio/user-guide/context-cache)可以显著提升响应速度，并降低使用成本。
* 批处理：通过合并多个请求为一个批次（同时合并相似请求或去除重复请求），可以减少请求次数，降低多次请求间的往返延迟，提高硬件利用率。百炼上提供了[批量推理（Batch）](https://help.aliyun.com/zh/model-studio/user-guide/batch-inference)API，通过利用空闲时间资源完成**离线推理任务**，你可以通过这些接口执行批量推理任务。

<br>

#### 2.1.3 减少 Tokens 的输入和输出

在让大模型处理任务过程中，减少 token 的输入和输出可以帮助模型缩减推理时间，从而加快响应速度，这对于实时性要求较高的应用场景（如对话系统、客服机器人）尤为重要。
* 输入端优化：精简输入内容，去除输入中的冗余信息或无关内容，只保留关键信息。例如，在对话系统中，可以通过预处理提取用户的意图和核心问题，而不是将整个对话历史输入模型。还可以对长文档或复杂输入内容生成摘要，作为模型的输入。这可以通过小型摘要模型或规则方法实现。
* 输出端优化：相对于输入端优化，输出端的优化往往更为重要，因为生成 token 的过程几乎总是最耗时的。最简单的优化方法就是通过提示词（prompt）明确要求模型生成简洁的回答。例如，“请用一句话回答”或“请非常简短地解释”。同时在结构化输出的场景中，可以尽可能优化输出内容，如去除重复性描述、缩短函数名等。最后虽不常见，你还可以在通过 API 调用模型时，明确指定最大输出长度（max_tokens 参数），限制生成内容的规模。

<br>

#### 2.1.4 并行化处理

在大模型应用中，采用并行化处理可以有效提升计算效率、缩短推理或训练时间。通过将任务分解为多个子任务（如数据并行、模型并行或流水线并行），可以分别在不同的GPU或服务器上同时执行，从而减少整体耗时。

例如，数据并行通过对输入数据分片并分配到多个设备上处理，模型并行将模型的不同层或参数分布到不同设备，而流水线并行则将计算过程划分为阶段依次执行。这些方法能够显著降低单设备的计算压力，突破内存限制，并提高吞吐量，为大模型的高效部署和运行提供关键支持。

<br>

#### 2.1.5 不要默认依赖大模型

大语言模型（LLM）虽然功能强大且用途广泛，但并不意味着它适合处理所有任务。在某些情况下，默认使用 LLM 可能会导致不必要的延迟或复杂性，而更简单、经典的方法反而能够提供更好的性能和效率。以下是一些优化建议：
* 硬编码：减少对动态生成的依赖。如果输出是高度标准化或受限的，硬编码可能是更好的选择，而不是依赖 LLM 动态生成内容。例如：
    * 操作确认消息：像“您的请求已成功提交”或“操作失败，请重试”等标准响应可以直接硬编码，无需 LLM 生成。
    * 拒绝消息：像“输入无效，请检查格式”等常见错误场景，可以预先定义多种变体并随机选择，既高效又避免了重复感。
* 预先计算：提前生成和复用内容。当输入选项有限时，可以通过预先计算生成所有可能的响应，并根据用户输入快速匹配。这种方法不仅能减少延迟，还能避免重复显示相同的内容。
* 利用经典 UI 组件：提升用户体验。在某些场景下，传统的 UI 组件比 LLM 生成的文本更能有效地传达信息。例如：
    * 汇总指标：使用图表、进度条或表格来展示数据，而不是让 LLM 生成一段描述性文字。
    * 搜索结果：通过分页、筛选器和排序功能呈现结果，比生成一段冗长的自然语言描述更直观。
* 传统优化技术：结合经典算法提升效率。即使是在 LLM 应用中，经典的优化技术依然适用。例如：
    * 二分查找：在处理有序数据时，使用二分查找快速定位目标，而不是让 LLM 遍历整个数据集。
    - 哈希映射：通过哈希表快速检索预定义的响应或模板，减少计算复杂度。

### 2.2 用户感知优化

除了上面介绍的几种提升 LLM 系统性能的原则外，还可以通过以下方法进一步提升用户在使用过程中的满意度。

#### 2.2.1 流式输出

将生成的内容逐步返回给用户的技术，可以减少用户感知延迟、提高交互的流畅性，从而显著提升用户体验，尤其是在实时性要求较高的场景（如在线客服、语音助手）中。如果你的应用架构中使用了负载均衡，可能需要关闭缓存和数据压缩功能，避免流式输出失效。

<img src="https://img.alicdn.com/imgextra/i2/O1CN01xHhaED1FT5vHNVzEi_!!6000000000487-1-tps-954-720.gif" width="500">

<br>

#### 2.2.2 分块处理

将任务分解为多个小块，分别处理并逐步返回结果。比如在 RAG 系统中，分块处理可以应用于检索和生成两个阶段：
* 将检索任务分解为多个子任务，例如按主题或数据源分块检索。
* 将生成任务分解为多个段落或句子，分别生成并返回。

<br>

#### 2.2.3 展示任务进度

可以让用户了解当前系统的处理状态，减少因未知等待带来的焦虑感。在 RAG 系统中，可以通过进度条、加载动画或文字提示展示任务进度。例如：
* 在前端显示一个进度条，实时更新检索完成的比例。
* 显示生成进度（如“正在生成回答... 已完成 3/5 句话”）。

<br>

#### 2.2.4 完善错误处理机制

这是确保系统稳定性和用户体验的关键。明确的错误处理机制不仅能够捕获和处理各种异常情况，还能通过返回友好的错误信息和重试建议，提升用户对系统的信任感和满意度。
* 分类错误和友好提示：根据问题分类返回精简结果或默认响应，其中友好的错误信息提示是非常有必要的，
    * 清晰易懂：避免使用技术术语，确保普通用户也能理解。
    * 具体明确：说明错误原因，并提供解决方案。
    * 情感友好：语气温和，避免让用户感到挫败。
* 重试机制和降级：
    * 自动重试：对于临时性错误（如网络抖动、服务短暂不可用），可以自动重试。注意设置最大重试次数和重试间隔，设置过大的重试次数和过短的重试间隔可能会增加额外的资源消耗。
    * 错误降级：对于每种错误类型，设计相应的降级方案。如在返回错误信息的同时，提供重试按钮或操作指南。

<br>

#### 2.2.5 提供用户反馈入口与持续改进

* 反馈入口：在界面中提供便捷的反馈渠道，鼓励用户提出意见或报告问题。
* 持续优化：通过分析用户反馈及用户行为数据，持续发现问题并优化。

<br>

## 3. 成本优化

### 3.1 优化系统性能时节约成本

在 **2.1 系统性能提升** 中介绍的很多方法不仅能降低延迟，提升性能，还能帮助你有效节约成本，包括：
* **用小模型替换大模型** ：不仅推理更快，同时也更便宜。
* **上下文缓存** ：高频重复的查询，将结果缓存起来，避免每次调用 LLM 的开销。（如通义千问系列模型 cache_token 单价为 input_token 单价的40%）
* **批量推理** ：通过合并或去重请求，减少不必要的模型调用。一般来说批量推理任务对时效要求不高，可以通过离线推理方式充分利用空闲计算资源进一步降低成本。（如百炼批量推理的计费仅为实时推理的50%）
* **减少 token 数量** ：降低计算资源的需求，从而节省硬件成本和能源消耗。
* **不让大模型处理所有任务** ：通过硬编码、预先计算等方式替换大模型推理可以有效降低成本。

<br>

上面这些方法比较通用，在实际业务部署中，通常有两种方式：自建环境和云上部署。自建环境初期成本高，且服务器采购流程长、维护难度大。相比之下，云上部署更适合初创企业及对成本敏感的业务。云部署将基础设施维护等任务交由云厂商处理，并能根据业务需求快速调整资源，实现高效利用。

接下来的内容将帮助你了解如何在云上选型并选择合适的计费方式。

<br>

### 3.2 云上部署成本优化

由于大模型应用系统通常涉及大规模数据存储、高性能计算和复杂的模型推理，其运行成本可能较高。以下将详细探讨如何在云上基于大模型应用进行架构选型和成本优化。

#### 3.2.1 选择合适的GPU实例规格

在阿里云上部署大模型时，需要根据模型的性能要求和预算限制综合选择合适的 ECS 实例类型（如单机单卡、单机多卡或多机多卡）。这不仅需要考虑模型本身的显存占用，还需要综合评估模型运行时的其他资源需求（如 KV Cache 的设置情况）：
* **模型参数量** ：模型的参数量直接决定了其显存需求。在前面微调章节中介绍过，1.5B 参数的模型通常需要约 5.59GB 显存（FP32 精度），而 DeepSeek-R1（满血版 671B）的存储至少需要 $\frac {671×{10^9}} {2^{30}}≈ 625GB$ 显存（FP8精度）。
* **KV Cache 占用** ：对于生成式任务（如文本生成或对话系统），KV Cache 是存储注意力机制中键值对缓存的空间。KV Cache 的大小与输入序列长度和批次大小成正比。例如，处理长上下文（如 2048 tokens）时，KV Cache 可能会占用大量显存。
* **精度设置** ：不同的计算精度（如 FP32、FP16）会影响显存需求。使用量化技术（如 INT8 或 INT4）可以显著减少显存占用。

##### **如何根据模型选择合适的GPU实例？**

以 DeepSeek-R1（671B）为例，模型本身预估占用显存 625GB（FP8 精度），KV Cache 使用 MLA（Multi-head Latent Attention） 优化，假设所有层都使用 MLA ，那么单卡下每个 token 预计占用的显存为 70KB，一个 64K token 长度请求在 8卡环境下 KV Cache 预估占用显存 35G，总共占用 660 GB，刚好可以选择 ecs.ebmgn8v.48xlarge 实例规格（实例显存 8*96 GB）。

##### **用户并发如何计算？**

还是以上面的例子来预估，可以处理的并发请求数和 Token 长度满受显存大小限制如下（不考虑内存对齐、ECC、框架运行时的显存占用）：

$\frac{req * l * (70 * 1024) }{2^{30}} + \frac{625}{8} < 96GB$

$req$表示并发请求数， $l$表示 token 长度。

假设 $l=64K$，可推算出 $req≈4$ 可见由于显存的局促导致同时服务客户的能力非常有限，如果需要承载更高并发可以适当限制单次请求的token长度或选择更高规格的GPU实例。

<br>

#### 3.2.2 选择合适的计费方式

确定GPU实例规格后，你要根据业务场景需求，选择合适的计费方式：

|  | [预付费（包年包月）](https://help.aliyun.com/zh/ecs/subscription) | [按量付费](https://help.aliyun.com/zh/ecs/pay-as-you-go-1) | [抢占式实例（spot）](https://help.aliyun.com/zh/ecs/preemptible-instance) |
|----|----|----|----|
| **说明** | 直接预支一笔费用，提前预留资源并享受更大的价格优惠。 | 按需开通和释放资源，无需提前支付费用，灵活快速。<br>短期运行成本较低，中长期使用资源可结合[节省计划](https://help.aliyun.com/zh/ecs/savings-plans)降低资源使用成本。 | 先使用后付费，市场价格根据供需关系实时变化，相对于按量付费最高能节约90%的成本。 |
| **推荐场景** | 适用于**相对稳定**的业务场景部署（如数月或数年）| 适用于**非固定时长**的业务场景部署 | 适用于非固定时长且**追求极致的成本控制**的业务场景部署，如非关键业务即使偶尔服务中断也不会造成重大损失。通常需结合重试机制或其他容错手段来应对短暂的服务不可用。 <br> `基于抢占式实例的在线推理服务部署方案：`[`PAI-EAS Spot最佳实践`](https://help.aliyun.com/zh/pai/use-cases/pai-eas-spot-best-practices) |

同时还建议你使用[预算管理](https://billing-cost.console.aliyun.com/expense-manage/expense-budget/list)设置预算告警，并定期通过[成本分析](https://billing-cost.console.aliyun.com/expense-manage/expense-analyze)识别高成本模块（如GPU/ECS实例）。测试不同配置的性价比，高负载时优先保障核心功能（如检索），关闭非必要服务（如复杂生成任务）。

<br>

## 4. 稳定性

前面内容中我们对于如何评估和优化模型性能，以及通过云服务进一步降低模型的运行成本有了比较深度的探讨，但是在将你的大模型应用投入生产时，必须保证系统能够提供稳定可靠的服务，这就像一家24小时营业的便利店，无论何时去都能正常购物。如果模型服务频繁“关门”或“反应迟钝”，用户会失去信任，企业也可能面临损失，通过以下方法可以有效保障你的模型应用的线上稳定性：

### 4.1 降低用户请求的资源消耗

在性能和成本优化中讲到的一些方法同样有助于提升系统稳定性，比如通过模型**小型化处理、异步批处理、缓存高频结果**等方式可以有效降低用户请求消耗的资源，从而使得相同配置的资源可以承担更多用户的请求，间接提升高并发场景下的稳定性。

<br>

### 4.2 自动化扩缩容

通用云应用的高可用架构同样适用于 LLM 系统，你可以根据用户请求数量自动增加或减少计算资源，在保证服务稳定的前提下避免资源浪费：
* **水平伸缩计算资源** ：[弹性伸缩（ESS）](https://help.aliyun.com/zh/auto-scaling/use-cases/deploy-a-highly-available-computing-cluster-using-a-balanced-distribution-strategy)动态调整ECS/GPU实例数量，或使用[函数计算（FC）](https://www.aliyun.com/product/fc)按需分配资源。
* **分散流量压力** ：通过[负载均衡（SLB）](https://www.aliyun.com/product/slb)提升高并发场景下的处理能力。可参考：[通过ALB将用户请求分配到多台ECS服务器](https://help.aliyun.com/zh/slb/application-load-balancer/getting-started/use-an-alb-instance-to-provide-ipv4-services)、[ALB添加函数计算FC作为后端服务](https://help.aliyun.com/zh/slb/application-load-balancer/use-cases/specify-a-function-from-function-compute-as-a-backend-server-of-alb)

<br>

### 4.3 评测基线管理

评测基线就像一把“标尺”，用来衡量模型的好坏，避免盲目优化后降低系统稳定性，同时也为后续的容灾降级（如切换至基线模型）提供可靠的后备方案。以下是建立评测基线的关键方法：
1. **建立基线模型**
    * **从简单开始** ：用基础算法（如决策树）或预设规则（如关键词匹配）作为初始模型，快速验证可行性。如：客服系统先用“关键词匹配”判断用户问题，准确率70%作为基线，后续优化需超过它。
    * **历史版本参考** ：将旧模型的性能设为最低门槛，新模型必须更好才能上线。
2. **定期测试与对比**
    * **时间维度** ：定期（如每周）用最新数据测试新旧模型，防止性能下降。如：电商推荐系统若点击率低于基线，说明模型需调整。
    * **场景维度** ：针对不同情况（如促销高峰期、不同地区）设置多组基线，全面评估稳定性。
3. **动态调整基线**
    * **应对数据变化** ：当业务或用户行为变化时，重新训练基线。如：金融风控政策调整后，需更新欺诈检测基线。
    * **适配业务需求** ：如更关注速度而非精度，改用轻量级模型（如蒸馏模型）作为新基线。
4. **融入自动化流程**
    * **自动拦截不合格模型** ：在部署流程中加入基线测试，未达标版本直接阻断。
    * **小范围试用新模型** ：先让5%用户试用新模型，效果达标再全面替换，即灰度发布。

<br>

### 4.4 模型实时监控与预警

* **关键指标看板** ：监控模型准确率、响应速度（如“每秒处理多少请求”）、错误率等，像“汽车仪表盘”一样实时显示健康状态。
* **数据漂移检测** ：对比当前输入数据与训练数据的分布差异。例如，若用户突然大量提问“如何退货”，系统会提示“数据分布异常，可能需重新训练模型”。
* **自动告警与日志追踪** ：设置阈值（如响应时间超过2秒将自动报警），并记录每条请求的输入、输出和上下文，方便快速定位问题。

<br>

### 4.5 容灾性设计

大模型应用的容灾设计可以想象成给系统准备“备用工具”和“应急计划”。良好的容灾设计可以应对突发故障，如模型响应异常、服务器宕机、网络中断或自然灾害（如地震）时，确保服务快速恢复：
* **降级与熔断机制**：当检测到模型响应异常（如准确率骤降或延迟超限），自动触发降级策略。如切换至备份模型（如上一稳定版本）或启用规则引擎兜底（如预设固定回复模板）。
* **通用应用容灾方案**同样适用于 LLM 系统：你可以跨地域跨可用区部署大模型应用，提升系统的可靠性。同时如果成本允许情况下，还可以创建备用环境，实现故障后的快速切换。如果对成本相对敏感，你的业务对系统恢复时间（如RTO和RPO）较为宽容，还可以通过快速创建环境的方式恢复业务。
* **定期演练预测试**：还建议你定期模拟故障场景（如模型异常、断网、服务器宕机等），验证容灾方案是否有效。

<br>


## 5. 安全与合规

### 5.1 安全合规的范围

结合本章前言提到的潜在问答风险场景和应用级安全考虑，RAG应用的安全合规主要包括：
* **内容安全合规** ：主要考虑输入输出合规检查、知识库访问控制。
* **应用服务安全** ：涉及到应用部署/访问、数据存储/传输等方面。

下面将重点讲解这两个层面的安全合规方案。

### 5.2 内容安全合规

我们先回顾一下智能答疑机器人的问答流程。问答流程主要包括用户、智能答疑机器人、知识库、大语言模型这四个主体。

一次常规的问答流程可概括为：用户发起提问，机器人根据用户问题从知识库检索出topK文本，并将用户问题和知识库文本结合起来，请求大模型生成回答，最后机器人将模型的回答返回给用户。

<img src="https://img.alicdn.com/imgextra/i3/O1CN01oHSxlr1EhSguiTa4G_!!6000000000383-2-tps-2128-1254.png" alt="问答流程" width="650"/>

涉及内容安全的关键阶段主要有：

- 输入阶段：用户发起提问。
- 输出阶段：机器人返回回答。
- 知识库召回阶段：从知识库中召回相关的topK文本。

针对RAG应用，内容安全合规检查方案的设计将围绕这三个阶段展开。

#### 5.2.1 方案概览
大模型的输入和输出包含的内容类型是相似的，常见的内容类型有：文本、图片、音频、视频等。在多模态场景下，输入和输出可能包含一种或多种内容类型。例如，当用户使用机器人进行报销咨询时，除报销文本描述外，还可能附加发票、火车票等图片。

为此，我们可以设计一套通用的合规检查机制，支持不同内容类型的检查，且适用于问答过程中的任意阶段。针对输入的内容合规检查，可将其放在用户提问后的阶段；而针对输出的内容合规检查，则应置于用户接收回答之前的阶段。特别地，我们需要在知识库召回阶段引入访问控制，以对召回的文本进行用户访问权限的过滤。整体方案流程见下图。

<img src="https://img.alicdn.com/imgextra/i1/O1CN019DfwwX2AAFgiTx5Ph_!!6000000008162-2-tps-2500-1220.png" alt="内容安全检查方案" width="800"/>

加入内容安全合规检查的问答流程包含八个阶段。

- **第一阶段**：用户发起提问。
- **第二阶段**：输入的内容合规检查。
    -  检查用户问题是否存在风险。
    - 如果通过，向机器人发起提问请求；如果不通过，立即返回，建议在页面向用户展示问题的风险点。
- **第三阶段**：根据用户问题检索知识库，并召回topK文本。 
- **第四阶段**：知识库访问控制。
    - 检查用户是否拥有召回文本的访问权限。
    - 过滤出用户拥有权限的文本，确保用户只能获取可访问的内容。
- **第五阶段**：机器人组合用户问题和知识库文本向大模型发送请求。
- **第六阶段**：机器人正常响应返回回答。    
- **第七阶段**：输出的内容安全合规检查。
    - 检查机器人回答是否存在风险。
    - 如果存在，过滤风险信息再返回给用户；如果不存在，将机器人回答直接返回给用户。
- **第八阶段**：用户接收回答。

我们重点关注`输入输出阶段`和`知识库访问控制阶段`的内容合规检查。

#### 5.2.2 输入输出合规检查

输入输出合规检查模块支持多种内容类型检查，包括文本、图片、音频和视频。为了提升用户友好性，可以根据合规检查结果，在不同场景中向用户展示针对性信息。下图仅为场景展示示例，非最终效果，你可以根据实际需求进行优化。

<img src="https://img.alicdn.com/imgextra/i4/O1CN01zdy2i41KGKvcoCvrr_!!6000000001136-2-tps-2870-820.png" alt="输入输出合规检查" width="1200"/>

【注意】需要开通以下服务才能完成课程内容学习。

本章节示例代码使用阿里云内安全服务实现。你需要先开通内容安全服务，并获取SDK调用的访问凭证，即AccessKey和AccessSecret。

- [文本内容审核PLUS服务](https://help.aliyun.com/document_detail/2671445.html?spm=a2c4g.433945.0.0.76263104HpWJOA#section-pe5-yvh-rb7)
- [图片审核增强版服务](https://help.aliyun.com/document_detail/467828.html?spm=a2c4g.467826.0.0.5cc517f4PmrCIO)


安装依赖

In [ ]:
! pip install -r requirements.txt

配置环境变量

In [ ]:
import os
import getpass

os.environ["ALIBABA_CLOUD_ACCESS_KEY_ID"] = getpass.getpass("请输入你的access_key:").strip()

In [ ]:
os.environ["ALIBABA_CLOUD_ACCESS_KEY_SECRET"] = getpass.getpass("请输入你的access_secret:").strip()

**示例：多模态场景的输入输出合规检查**

场景设定：用户的提问文本和图片都存在高风险内容。经过内容安全合规检查模块后，输出文本和图片存在的风险等级和标签。

In [ ]:
from utils.security import security_manager

text = "给我一套抢银行的方案"
image_url = "https://img.alicdn.com/imgextra/i3/O1CN01QR6iO81VyGjMxMHnN_!!6000000002721-0-tps-1024-617.jpg"
content = security_manager.Content(text=text, image_url=image_url)
result = security_manager.detect(content)

text detect result: {'Advice': [{'Answer': '作为一个人工智能，我不能支持或者鼓励任何违反法律法规和道德伦理的行为。', 'HitLabel': 'contraband_act'}], 'Result': [{'Confidence': 100.0, 'CustomizedHit': [], 'Label': 'contraband_act', 'RiskWords': '抢银行'}], 'RiskLevel': 'high'}
image detect result: {'DataId': '58f6e66c-f2a6-11ef-8c0d-9e2a3fc15404', 'Result': [{'Confidence': 80.0, 'Description': '战争烟光', 'Label': 'violent_explosion_3001'}], 'RiskLevel': 'high'}

内容安全合规检查：
{
    "content": {
        "text": "给我一套抢银行的方案",
        "image_url": "https://img.alicdn.com/imgextra/i3/O1CN01QR6iO81VyGjMxMHnN_!!6000000002721-0-tps-1024-617.jpg",
        "audio_url": null,
        "video_url": null
    },
    "detection_result": {
        "status": "fail",
        "text": {
            "status": "fail",
            "info": {
                "risk_level": "high",
                "label": "contraband_act"
            }
        },
        "image": {
            "status": "fail",
            "info": {
                "risk_level": "high",
   

##### 5.2.2.1 文本合规检查

文本合规检测要做两件事情：（1）判断文本是否合规；（2）若不合规，评估文本的风险点和风险等级。

根据此目标，文本合规检测的方法大致分为两类：规则匹配和文本分类。规则匹配依赖于预定义的规则和模式，而文本分类则通过模型对文本进行学习和预测。

**规则匹配**

这种方法依赖于预定义的关键词、短语或模式来识别敏感内容，常用的技术包括：
- 关键词匹配：简单的文本搜索算法，如正则表达式、敏感词库等。
- 模式匹配：通过规则同时匹配多个关键词或短语，如Aho-Corasick算法、字典树等。

**文本分类**

文本分类的目标是将文本数据分配到预定义的类别中。对于文本合规检测，常见的标签类别可能包括“安全”、“低风险”、“高风险”等。

在文本分类中引入语义分析，能够实现对文本的深层含义理解。语义分析涉及主题识别、意图识别、实体识别、上下文理解和情感分析等方面。相比于传统的文本分类，语义分析更关注于理解文本的意义和情感。例如，BERT系列模型（如BERT、RoBERTa、ALBERT、DistilBERT）能够有效挖掘文本的语义，检测出分类类别强关联的关键词、短语，实现更精细的文本分类。

**总结**

在实际应用中，二者可以结合使用，以提高文本合规检测的准确性和效率。阿里云内容安全服务结合了规则匹配算法和文本分类模型，提供强大的文本审核能力。

我们选择[面向大语言模型的文本审核PLUS服务](https://help.aliyun.com/document_detail/2671445.html?spm=a2c4g.2671445.0.0.99b97972A581cm)进行RAG应用的文本安全合规检查。

如果您想直接体验文本审核能力，可以通过阿里云API官网发起调用，前往[文本审核API页面](https://api.aliyun.com/api/Green/2022-03-02/TextModeration?RegionId=cn-shanghai)。

**示例：文本合规检查（带私域数据的规则匹配）**

场景设定：我们教育公司的内网域名属于敏感信息，员工在使用智能答疑机器人提问时，不允许提及带有 jiaoyu.com 后缀的内部域名。要求合规检查模块能够识别内网域名信息，进行有效拦截。

In [ ]:
from utils.security import text_security

text = "公司OA域名oa.jiaoyu.com预发环境访问链接是什么"

# 模型类型
# llm_query_moderation：用于大语言模型输入指令的审核
# llm_response_moderation：用于大语言模型生成文字的审核
result = text_security.detect(text, model="llm_query_moderation")
print('text detect result: {}'.format(result))

text detect result: {'Advice': [], 'Result': [{'CustomizedHit': [], 'Label': 'nonLabel'}], 'RiskLevel': 'none'}


根据上述代码运行结果可以看到，大模型并没有按照预期检测出带有jiaoyu.com后缀的域名。

大模型为什么没能检测到呢？这是因为教育公司内网域名属于私域数据。默认情况下，阿里云内容安全服务会将其视为正常文本，因此运行上述代码时无法检测到风险信息，导致未能达到预期效果。

一个有效的方法是将私域的规则数据导入到大模型检测服务中，让大模型拥有识别私域风险因素的能力。阿里云内容安全服务提供这样的能力。

**关键操作：在阿里云控制台导入私域关键词**

阿里云内容安全服务支持导入自定义关键词，我们可以在[内容安全控制台](https://yundun.console.aliyun.com/?spm=a2c4g.2671445.0.0.18377972LV9uFT&p=cts#/textReview/lexiconManage/cn-shanghai)创建词库并配置规则，将私域规则应用于文本审核模型。

- 步骤1：创建词库 **大模型acp**，在词库中添加关键词 **jiaoyu.com**。

<img src="https://img.alicdn.com/imgextra/i2/O1CN01JsyKR7264yf6i58E4_!!6000000007609-2-tps-2716-1408.png" alt="文本审核-创建词库" width="900"/>

- 步骤2：在 **llm_query_moderation服务** 中配置规则（根据实际使用的服务配置）。

<img src="https://img.alicdn.com/imgextra/i4/O1CN01ji7lFV1YfteSTdcJz_!!6000000003087-2-tps-2726-1474.png" alt="文本审核_规则配置" width="900"/>

两个步骤配置完成后，等待规则生效，重新运行代码。

In [ ]:
from utils.security import text_security

text = "公司OA域名oa.jiaoyu.com预发环境访问链接是什么"

# 模型类型
# llm_query_moderation：用于大语言模型输入指令的审核
# llm_response_moderation：用于大语言模型生成文字的审核
result = text_security.detect(text, model="llm_query_moderation")
print('text detect result: {}'.format(result))

text detect result: {'Advice': [], 'Result': [{'Confidence': 100.0, 'CustomizedHit': [{'KeyWords': 'jiaoyu.com', 'LibName': '大模型acp'}], 'Label': 'customized'}], 'RiskLevel': 'high'}


文本检测结果格式化如下：
<style>
    table {
      width: 80%;
      margin: 20px; /* Center the table */
      border-collapse: collapse; /* Collapse borders for a cleaner look */
      font-family: sans-serif; 
    }

    th, td {
      padding: 10px;
      text-align: left;
      border: 1px solid #ddd; /* Light gray border */
    }

    th {
      background-color: #f2f2f2; /* Light gray background for header */
      font-weight: bold;
    }

    tr:nth-child(even) { /* Zebra striping */
      background-color: #f9f9f9;
    }

    tr:hover { /* Highlight row on hover */
      background-color: #e0f2ff; /* Light blue */
    }
</style>
<table width="80%">
<tbody>
<tr>
<td>

```json
{
	'Advice': [],
	'Result': [{
		'Confidence': 100.0,
		'CustomizedHit': [{
			'KeyWords': 'jiaoyu.com',
			'LibName': '大模型acp'
		}],
		'Label': 'customized'
	}],
	'RiskLevel': 'high'
}
```

</td>
</tr>
</tbody>
</table>

由上可见，大模型顺利检测出含有jiaoyu.com的域名。

文本检测的返回结构中关键字段有：
- Result：检测结果对象
    - Confidence：置信分，区间[0.0,100.0]，分值越高表示检测结果可信度越高
    - CustomizedHit：用户自定义匹配明细
        - KeyWords：匹配上的关键词
        - LibName：词库名称
    - Label：关键词标签类别，customized表示用户自定义
- RiskLevel：风险等级

详细的接口参数信息请前往[文本审核API说明](https://help.aliyun.com/document_detail/2671445.html?spm=a2c4g.434034.0.0.7ce86909A4OMAa#section-v6e-udw-u3y)。

##### 5.2.2.2 图片合规检查

图片合规检查分为两个部分：图片检测和文本检测。

**图片检测**

关注图像内容本身的合规性，包括：
- 图片内容检测：可使用卷积神经网络等深度学习模型对合规性（如暴力、色情、仇恨言论等）进行分类。
- 敏感物体检测：检测图片中的敏感物体，如武器、毒品、色情内容等。目标检测经典算法有：YOLO系列（YOLOv3/YOLOv4/YOLOv5等）、Faster R-CNN等。
- 版权检查：利用图像指纹识别技术（如 PHash）检测相似图像，避免使用未经授权的内容。
- 水印和品牌标志检查：检测图像中是否存在水印或者品牌标志

**文本检测**

关注图像中包含的文字内容，包括：
- 从图片中提取文本。通常是使用光学字符识别（OCR）技术提取文字信息，如Tesseract算法等。
- 文本合规检测。具体方案参考上一小节。

阿里云内容安全服务提供图片审核能力，我们选择[图片审核增强版服务](https://help.aliyun.com/document_detail/467826.html?spm=a2c4g.467826.0.0.329829b16I0AXR)进行代码示例。

如果您想直接体验图片审核能力，可以通过阿里云API官网发起调用，前往[图片审核API页面](https://api.aliyun.com/api/Green/2022-03-02/ImageModeration)。

**示例：图片合规检查**

In [ ]:
from utils.security import image_security

image_url = "https://img.alicdn.com/imgextra/i3/O1CN01QR6iO81VyGjMxMHnN_!!6000000002721-0-tps-1024-617.jpg"

# 检测服务 baselineCheck_pro：通用基线检测_专业版
result = image_security.detect(image_url)
print('image detect result: {}'.format(result))

image detect result: {'DataId': 'a53ddb3e-f2a6-11ef-8c0d-9e2a3fc15404', 'Result': [{'Confidence': 80.0, 'Description': '战争烟光', 'Label': 'violent_explosion_3001'}], 'RiskLevel': 'high'}


图片检测结果格式化如下：
<style>
    table {
      width: 80%;
      margin: 20px; /* Center the table */
      border-collapse: collapse; /* Collapse borders for a cleaner look */
      font-family: sans-serif; 
    }

    th, td {
      padding: 10px;
      text-align: left;
      border: 1px solid #ddd; /* Light gray border */
    }

    th {
      background-color: #f2f2f2; /* Light gray background for header */
      font-weight: bold;
    }

    tr:nth-child(even) { /* Zebra striping */
      background-color: #f9f9f9;
    }

    tr:hover { /* Highlight row on hover */
      background-color: #e0f2ff; /* Light blue */
    }
</style>
<table width="80%">
<tbody>
<tr>
<td>

```json
{
	'DataId': 'a53ddb3e-f2a6-11ef-8c0d-9e2a3fc15404',
	'Result': [{
		'Confidence': 80.0,
		'Description': '战争烟光',
		'Label': 'violent_explosion_3001'
	}],
	'RiskLevel': 'high'
}
```

</td>
</tr>
</tbody>
</table>

我们选择的图片审核服务是baselineCheck_pro（通用基线检测_专业版），支持识别不同国家的国旗。根据上面检查结果可见，大模型检查出图片是其他国家国旗，并给出图片标签类别和风险等级。

图片检测的返回结构中关键字段有：
- Result：检测结果对象
    - Confidence：置信分，区间[0.0,100.0]，分值越高表示检测结果可信度越高
    - Description：图片标签描述
    - Label：图片标签类别
- RiskLevel：风险等级

详细的接口参数信息请前往[图片审核API说明](https://help.aliyun.com/document_detail/467829.html?spm=a2c4g.467828.0.0.549117f4h4vhr8#section-9c5-lr2-oqe)。

##### 5.2.2.3 音频合规检查

音频合规检查包括纯音频检查和音频转文本合规检测。

**纯音频检查**

该部分关注音频信号的特征和内容，如频率、音调、音量及特定音频片段，常用于检测音乐、音效及其他非语言内容的合规性。
常用的音频分析框架包括：
- Librosa：Python库，提供音频分析功能，如特征提取、音频效果处理和节拍检测等。
- Essentia：C++和Python库，包含丰富的音频特征提取工具，如音高、和声和节奏等，适用于合规检测。
- PyDub：简单易用的Python库，适合进行音频处理和基本分析。
- Aubio：专注于音高检测和音频事件检测的工具。

**音频转文本合规检测**

该部分关注音频中的语言内容，将音频转换为文本来检测合规性，适用于监测敏感词和违规语言等情境。通常使用自动语音识别（ASR）技术将音频信号转换为文本，再对文本进行合规检测。

阿里云内容安全服务提供音频审核能力，我们选择[音频审核增强版服务](https://help.aliyun.com/document_detail/604968.html?spm=a2c4g.604967.0.0.406f17f4DdWtYs)进行代码示例。

如果您想直接体验音频审核能力，可以通过阿里云API官网发起调用，前往[音频审核API页面](https://api.aliyun.com/api/Green/2022-03-02/VoiceModeration)。

**示例：音频合规检查**

参考文本审核的操作，我们在[内容安全控制台](https://yundun.console.aliyun.com/?spm=a2c4g.2671445.0.0.18377972LV9uFT&p=cts#/textReview/lexiconManage/cn-shanghai)的词库 **大模型acp** 中增加关键词，并应用于音频审核模型。

- 步骤1：在词库 **大模型acp** 中添加关键词 **密码** 。

<img src="https://img.alicdn.com/imgextra/i3/O1CN01glQ0By20vICx6sZhD_!!6000000006911-2-tps-3022-1632.png" alt="音频审核-配置词库" width="900"/>

- 步骤2：在 **audio_media_detection服务** 中配置规则（根据实际使用的服务配置）。

<img src="https://img.alicdn.com/imgextra/i2/O1CN01wHvO7S27PsLi8OnDt_!!6000000007790-2-tps-3016-1614.png" alt="音频审核_规则配置" width="900"/>

两个步骤配置完成后，等待规则生效，运行下面代码。

In [ ]:
from utils.security import audio_security
import time

audio_url = 'https://help-static-aliyun-doc.aliyuncs.com/file-manage-files/zh-CN/20241016/whwtft/acp_secret.mp3'
# 提交检测任务
# 音频审核服务：audio_media_detection
task_id = audio_security.submit_task(audio_url)
# 等待5秒再查询
time.sleep(5)
# 根据任务id查询结果
audio_security.get_result(task_id)

audio submit task:{'TaskId': 'au_f_eyCjrLm2VgR8OYf8TpeMqP-1ACXZ8'}
audio detect result:{'SliceDetails': [{'EndTime': 2, 'EndTimestamp': 1728438939994, 'Extend': '{"customizedWords":"密码","customizedLibs":"大模型acp"}', 'Labels': 'C_customized', 'StartTime': 0, 'StartTimestamp': 1728438937994, 'Text': '登录账号密码是一二三四五六', 'Url': 'http://oss-cip-shanghai.oss-cn-shanghai.aliyuncs.com/cip-media/voice/26193ec34/847846_as00000.wav?Expires=1728460541&OSSAccessKeyId=LTAI5t6KYjFmqpFzteQWRo4j&Signature=SYen1fOpsyhMBjuJhYuPq%2BFpUtY%3D'}], 'TaskId': 'au_f_eyCjrLm2VgR8OYf8TpeMqP-1ACXZ8'}


音频检测结果格式化如下：

<style>
    table {
      width: 80%;
      margin: 20px; /* Center the table */
      border-collapse: collapse; /* Collapse borders for a cleaner look */
      font-family: sans-serif; 
    }

    th, td {
      padding: 10px;
      text-align: left;
      border: 1px solid #ddd; /* Light gray border */
    }

    th {
      background-color: #f2f2f2; /* Light gray background for header */
      font-weight: bold;
    }

    tr:nth-child(even) { /* Zebra striping */
      background-color: #f9f9f9;
    }

    tr:hover { /* Highlight row on hover */
      background-color: #e0f2ff; /* Light blue */
    }
</style>
<table width="80%">
<tbody>
<tr>
<td>

```json
{
	'SliceDetails': [{
		'EndTime': 2,
		'EndTimestamp': 1728438939994,
		'Extend': '{"customizedWords":"密码","customizedLibs":"大模型acp"}',
		'Labels': 'C_customized',
		'StartTime': 0,
		'StartTimestamp': 1728438937994,
		'Text': '登录账号密码是一二三四五六',
		'Url': 'http://oss-cip-shanghai.oss-cn-shanghai.aliyuncs.com/cip-media/voice/26193ec34/847846_as00000.wav?Expires=1728460541&OSSAccessKeyId=LTAI5t6KYjFmqpFzteQWRo4j&Signature=SYen1fOpsyhMBjuJhYuPq%2BFpUtY%3D'
	}],
	'TaskId': 'au_f_eyCjrLm2VgR8OYf8TpeMqP-1ACXZ8'
}
```

</td>
</tr>
</tbody>
</table>

音频审核是异步执行的，需要先提交任务再等待几秒才能获取检测结果。根据上面的运行结果可见，大模型将音频转为了文字并进行了检测。

音频检测的返回结构中关键字段有：
- Labels：音频标签类别，C_customized表示用户导入的自定义标签
- Extend：自定义标签的扩展字段
    - customizedWords：匹配的关键词
    - customizedLibs：词库
- Text：音频文本内容

详细的接口参数信息请前往[音频审核API说明](https://help.aliyun.com/document_detail/604973.html?spm=a2c4g.604972.0.0.7e853a9dXiShnN#section-m14-l36-468)。

##### 5.2.2.4 视频合规检查

视频合规检测是一个复杂的过程，包含四个关键步骤：
- 视频预处理：格式转换、视频分段、帧提取。
- 图片合规检测：视频中的图像内容符合规定，避免出现敏感或违规图像。
- 文本合规检测：审查视频中的文字信息，包括字幕和音频转录内容。
- 音频合规检测：确保视频中的音频元素符合合规要求，避免版权和内容违规问题。

综合上述四个步骤，视频合规检测流程能够有效识别和过滤不合规内容，用来保障视频的健康性和合规性。我们可以利用多种开源工具和库（如FFmpeg、OpenCV、TensorFlow 等）来搭建一个完整的视频合规检查服务，减少人工审核的负担。

阿里云内容安全服务提供视频审核能力，我们选择[视频审核增强版服务](https://help.aliyun.com/document_detail/2505807.html?spm=a2c4g.2505806.0.0.5721ce907KX0zV)进行代码示例。

如果您想直接体验图片审核能力，可以通过阿里云API官网发起调用，前往[视频审核API页面](https://api.aliyun.com/api/Green/2022-03-02/VideoModeration)。

**示例：视频合规检查**

上面音频审核配置中我们已经在词库添加了关键词 **密码** ，视频审核只需要在[内容安全控制台](https://yundun.console.aliyun.com/?spm=a2c4g.2671445.0.0.18377972LV9uFT&p=cts#/videoReview/ruleConfig/config/videoDetection/undefined/cn-shanghai)进行规则配置即可。

<img src="https://img.alicdn.com/imgextra/i4/O1CN01PzbsoN1Xoli0ToU69_!!6000000002971-2-tps-3010-1630.png" alt="视频审核-规则配置" width="900"/>

配置完成后，等待规则生效，运行下面代码。

In [ ]:
from utils.security import video_security
import time

video_url = 'https://help-static-aliyun-doc.aliyuncs.com/file-manage-files/zh-CN/20241105/bbsokg/acp_secret.mp4'
# 提交检测任务
task_id = video_security.submit_task(video_url)
# 等待20秒再查询
time.sleep(20)
# 根据任务id查询结果
video_security.get_result(task_id)

video submit task:{'TaskId': 'vi_f_J00HYR8x3P66IubIwSrQyh-1ALNs3'}
video detect result:{'Code': 200, 'Data': {'AudioResult': {'AudioSummarys': [{'Label': 'C_customized', 'LabelSum': 1}], 'SliceDetails': [{'EndTime': 2, 'EndTimestamp': 1730810748526, 'Extend': '{"customizedWords":"密码","customizedLibs":"大模型acp"}', 'Labels': 'C_customized', 'StartTime': 0, 'StartTimestamp': 1730810746526, 'Text': '登录密码是一二三四五六 ', 'Url': 'http://oss-cip-shanghai.oss-cn-shanghai.aliyuncs.com/cip-media/voice/051ec34/28375_as00000.wav?Expires=1730832364&OSSAccessKeyId=LTAI5t6KYjFmqpFzteQWRo4j&Signature=34E3PVSYIYpE2oUpoBPj6DzBrZ0%3D'}]}, 'FrameResult': {'FrameNum': 0, 'FrameSummarys': [], 'Frames': []}, 'TaskId': 'vi_f_J00HYR8x3P66IubIwSrQyh-1ALNs3'}, 'Message': 'SUCCESS', 'RequestId': '20FA1F94-4BB2-53E9-9470-6211CCD24BAE'}


视频检测结果格式化如下：

<style>
    table {
      width: 80%;
      margin: 20px; /* Center the table */
      border-collapse: collapse; /* Collapse borders for a cleaner look */
      font-family: sans-serif; 
    }

    th, td {
      padding: 10px;
      text-align: left;
      border: 1px solid #ddd; /* Light gray border */
    }

    th {
      background-color: #f2f2f2; /* Light gray background for header */
      font-weight: bold;
    }

    tr:nth-child(even) { /* Zebra striping */
      background-color: #f9f9f9;
    }

    tr:hover { /* Highlight row on hover */
      background-color: #e0f2ff; /* Light blue */
    }
</style>
<table width="80%">
<tbody>
<tr>
<td>

```json
{
	'Code': 200,
	'Data': {
		'AudioResult': {
			'AudioSummarys': [{
				'Label': 'C_customized',
				'LabelSum': 1
			}],
			'SliceDetails': [{
				'EndTime': 2,
				'EndTimestamp': 1730810748526,
				'Extend': '{"customizedWords":"密码","customizedLibs":"大模型acp"}',
				'Labels': 'C_customized',
				'StartTime': 0,
				'StartTimestamp': 1730810746526,
				'Text': '登录密码是一二三四五六 ',
				'Url': 'http://oss-cip-shanghai.oss-cn-shanghai.aliyuncs.com/cip-media/voice/051ec34/28375_as00000.wav?Expires=1730832364&OSSAccessKeyId=LTAI5t6KYjFmqpFzteQWRo4j&Signature=34E3PVSYIYpE2oUpoBPj6DzBrZ0%3D'
			}]
		},
		'FrameResult': {
			'FrameNum': 0,
			'FrameSummarys': [],
			'Frames': []
		},
		'TaskId': 'vi_f_J00HYR8x3P66IubIwSrQyh-1ALNs3'
	},
	'Message': 'SUCCESS',
	'RequestId': '20FA1F94-4BB2-53E9-9470-6211CCD24BAE'
}
```

</td>
</tr>
</tbody>
</table>

由上可见，视频检测结果包含音频检测结果和视频帧检测结果。音频检测结果包含检测标签和文本匹配，由于我们测试的视频没有合规风险，因此视频帧没有检测结果，符合预期。

视频检测的返回结构中关键字段有：
- AudioResult：音频检测结果
- FrameResult：视频检测结果

详细的接口参数信息请前往[视频审核API说明](https://help.aliyun.com/document_detail/2505810.html?spm=a2c4g.2505806.0.0.23663a9dQBUD4Z#Yy25U)。

#### 5.2.3 知识库访问控制

根据用户的问题，从知识库召回的相关文本需要进行访问控制，确保仅返回用户拥有权限的内容。

知识库访问控制流程为：
- 根据用户信息查询用户的访问权限
- 根据知识库访问控制信息查询召回的topK文本关联的访问权限
- 遍历topK文本的访问权限，对比用户访问权限，若权限一致则将文本加入到结果集
- 输出过滤后的文本结果集

<img src="https://img.alicdn.com/imgextra/i1/O1CN01O79j7z1MzL4qJ7gV9_!!6000000001505-2-tps-1112-800.png" alt="知识库访问控制流程" width="550"/>


**示例：使用csv模拟权限存储实现知识库访问控制**

场景设定：每位教育公司的员工有唯一的工作职位，如普通员工、经理等。工作职位对应的查看知识库的内容权限不同，例如普通员工只能查看自己职位的薪酬方案，不允许查看上级领导的薪酬方案，但上级领导可以查看下属的薪酬方案。

In [ ]:
from utils.security.kb_access_control import kb_filter

# 请在utils.security.kb_access_control/db/user.csv 查看user_id
# 根据用户id查询拥有权限的召回文本
user_id = 201
kb_filter.get_filter_contents(user_id)

当前用户的职位: employer

==========召回文本==========
普通员工薪酬方案：xxxx
团队leader的薪酬方案：xxxx
部门经理的薪酬方案：xxxx
公司薪酬方案总体来说分为这几个部分。
除了上述固定薪酬方案外，公司鼓励大家积极参与国内外会议，积极发表专利和论文，也有相应的激励。
==========召回文本==========

用户拥有权限的召回文本: ['普通员工薪酬方案：xxxx', '公司薪酬方案总体来说分为这几个部分。']


### 5.3 应用服务安全

模型应用服务安全主要包括：模型应用部署平台的安全、数据传输安全以及知识库数据存储的安全。阿里云提供了全面的保障体系，涵盖应用部署、访问、数据传输和存储等。你无需担心应用服务的安全问题，可以安心专注于搭建和优化你的模型应用。

<img src="https://img.alicdn.com/imgextra/i1/O1CN01cUFGSw1VFNsV8X5Qu_!!6000000002623-2-tps-2846-832.png" alt="应用级安全保障体系" width="1200"/>

#### 5.3.1 应用部署平台安全

应用部署平台的安全性是确保大模型在生产环境中稳定运行、保护用户数据隐私和防止恶意攻击的关键。由于大模型通常处理敏感数据（如用户输入、推理结果等），并且可能面临复杂的攻击场景（如对抗攻击、注入攻击等），部署平台需要从多个层面构建全面的安全防护体系。

你可以将模型应用托管到阿里云的部署平台，如ECS、PAI-EAS平台和函数计算平台。部署平台支持阿里云全平台的RAM账号管理，支持操作审计。此外，不同部署平台还有各自的操作权限控制和网络安全配置：
* **权限控制**
    * **最小权限原则** ：使用阿里云 RAM（资源访问管理）为不同用户分配最小权限。定期审查权限分配，清理不必要的权限。
    * **多因素认证（MFA）** ：强制管理员账户启用 MFA，增强登录安全性。
* **网络安全配置**
    * **云防火墙** ：配置入站和出站规则，仅允许必要的流量通过。限制 SSH 和 RDP 访问，仅允许特定 IP 地址访问管理接口。
    * **安全组规则** ：为每个服务配置最小权限原则，例如只开放模型服务端口。
    * **DDoS攻击防护** ：限制API调用频率

<br>

#### 5.3.2 数据传输安全

通常大模型应用需要加密通信，从而确保数据在传输过程中不被窃取、篡改或泄露。具体实现方式可以参考：[以加密的方式接入模型推理功能](https://help.aliyun.com/zh/model-studio/user-guide/encrypted-access-to-model-inference-functions)

在通过阿里云百炼平台开发与构建大模型应用过程中，百炼不会存储模型请求和响应数据，Prompt在传输过程中严格加密。你可以通过阿里云账号搭建VPC专网通道安全调用百炼API，确保模型应用可靠访问大模型，详见：[通过终端节点私网访问阿里云百炼平台](https://help.aliyun.com/zh/model-studio/user-guide/access-model-studio-through-privatelink)。此外，百炼平台还结合了网关管理、RAM账号控制、内部审计和模型加密等多重安全策略，全方位保障你的应用安全。

<br>

#### 5.3.3 知识库数据存储安全

在RAG应用中，知识库向量数据、日志存储、中间处理数据和多模态数据均归属于你的阿里云账号。为了保障用户账号的数据安全，阿里云提供了KMS加密、SDDP防护等安全措施。
数据安全是大模型应用的核心，以下是一些措施：
* **数据加密** ：使用阿里云 KMS（密钥管理服务）对敏感数据进行加密存储。
* **敏感数据防护** ：对敏感字段（如身份证号、手机号）进行脱敏处理。你可以通过[阿里云数据安全中心](https://www.aliyun.com/product/security/sddp)，检测数据库、OSS、SLS中的敏感信息。
* **备份与恢复** ：制定备份策略（如异地备份机制），周期性自动备份数据库，并将备份文件加密存储。

<br>

### 5.4 大模型应用备案

自2023年8月15日起，《生成式人工智能服务管理暂行办法》正式生效，监管部门要求对AIGC（生成式人工智能）相关的APP和小程序进行整改和合规备案，要求：
* 已上架但未完成合规手续的应用将被下架
* 未上架的应用必须完成合规动作后方可上线

阿里云百炼大模型服务平台为云上企业提供技术支持，涵盖合规要求的算法备案等内容，详情参见[大模型开发应用上架应用商店/微信小程序申请指南](https://help.aliyun.com/zh/model-studio/support/merchant-application-guide?spm=a2c4g.2400264.0.i1)。

## ✅ 本节小结

在本节课程中，我们学习了以下内容：
- **大模型应用发布至生产环境的关键要素** ，包括业务需求分析、模型的功能性需求（如自然语言处理、视觉、语音等场景的模型选择）与非功能性需求（性能、成本、稳定、安全等）
- **性能与成本优化策略** ，如模型压缩（剪枝、量化、知识蒸馏）、请求批处理、缓存机制、输入输出Token优化及并行化处理
- **用户体验提升方法** ，包括流式输出、分块处理、任务进度展示、错误处理机制及用户反馈闭环设计
- **稳定性提升方法** ，包括自动化扩缩容、模型实时监控与预警以及通过容灾设计保障业务稳定运行
- **云上架构选型与安全合规** ，涵盖GPU实例规格选择、多卡推理优化、数据传输加密、内容安全审核（文本/图片/音频/视频）及知识库访问控制

除了本节课程中的示例展示的内容之外，你还可以尝试：
- **多卡PD分离提升模型推理效率** ：在大模型（如语言模型）的推理过程中，Prefill（预填充阶段） 和 Decode（解码阶段） 是两个关键阶段，分别对应输入处理和输出生成的过程。在多卡环境下，通过分解 Prefill 和 Decode 过程可以优化模型的推理速度，从而满足 TTFT（首 Token 延迟） 和 TPOT（每 Token 生成时间） 的 SLO 需求。[DistServe](https://github.com/LLMServe/DistServe) 是分离 Prefill/Decode 的典型系统实现，结合分块预填充和异步流水线，显著提升吞吐并降低延迟。
- **实现多模态安全审核** ：扩展视频合规检测流程，集成自定义敏感词库与版权识别。
- **引入 MLOps 管理 AI 系统** ：MLOps（Machine Learning Operations）是机器学习与 DevOps 的结合，旨在通过自动化、标准化和协作的方式管理机器学习模型的全生命周期。它借鉴了软件工程中的 DevOps 理念，将开发（Development）和运维（Operations）紧密结合，确保机器学习模型能够高效地从开发阶段过渡到生产环境，并在生产环境中持续优化和迭代。

在学习本课程的时候，如果你想了解更多相关概念和原理，可以尝试进一步学习如下内容或通过大模型给出学习建议，如：
- **KV Cache优化实践** ：深入理解KV Cache的作用及优化方法。
- **安全合规框架** ：结合《[生成式人工智能服务管理暂行办法](https://www.gov.cn/zhengce/zhengceku/202307/content_6891752.htm)》，设计符合AIGC备案要求的RAG应用架构。

<style>
    table {
      width: 80%;
      margin: 20px; /* Center the table */
      border-collapse: collapse; /* Collapse borders for a cleaner look */
      font-family: sans-serif; 
    }

    th, td {
      padding: 10px;
      text-align: left;
      border: 1px solid #ddd; /* Light gray border */
    }

    th {
      background-color: #f2f2f2; /* Light gray background for header */
      font-weight: bold;
    }

    tr:nth-child(even) { /* Zebra striping */
      background-color: #f9f9f9;
    }

    tr:hover { /* Highlight row on hover */
      background-color: #e0f2ff; /* Light blue */
    }
</style>


## 🔥 课后小测验

【单选题】以下哪个代码片段最能体现“输入输出的内容安全合规检查”的流程？（ ）

<table width="80%">
<tbody>
<tr>
<td>
A.
</td>
<td>

```python
def process_input(user_input):
    response = model(user_input)
    check_result = security_check(response)
    if check_result:
        return response
    else:
        return "回答包含不合规信息"
```
</td>
</tr>
<tr>
<td>
B.
</td>
<td>

```python
def process_input(user_input):
    check_result = security_check(user_input)
    if check_result:
        response = model(user_input)
        check_result = security_check(response)
        if check_result:
          return response
        else:
          return "回答包含不合规信息"
    else:
        return "提问包含不合规信息"
```
</td>
</tr>
<tr>
<td>
C.
</td>
<td>

```python
def process_input(user_input):
    response = model(user_input)
    return response
```
</td>
</tr>
<tr>
<td>
D.
</td>
<td>

```python
def process_input(user_input):
    check_result = security_check(user_input)
    return check_result
```
</td>
</tr>
</tbody>
</table>


答案：B

解释： 输入输出的内容安全合规检查应该在模型处理之前对用户输入进行检查，并且在模型处理之后对模型生成结果进行检查。选项B首先对用户输入 user_input 进行安全检查，只有通过检查才会将输入传递给模型 model 处理，然后对模型的输出结果 response 进行安全检查。其他选项要么在模型处理后进行检查 (A)，要么完全没有检查 (C)，要么只返回检查结果而不处理模型调用 (D)。

<br>

【多选题】你正在构建一个系统，用于检测用户上传图片中的违规文字信息。以下哪些技术或方法是必要的？（ ）

A. OCR (光学字符识别)

B. 文本合规检测

C. 使用多模态embedding模型获取图片的向量

D. 使用通义万相模型进行检测

E. 音频转文本

F. PHash 图像指纹识别

答案：AB

解释： A. OCR 用于从图片中提取文本信息，B. 文本合规检测用于分析提取的文本是否违规。C. 多模态embedding模型 用于多模态检索，D. 通义万相 用于生成图片，F. PHash 用于版权检查，都与检测违规文字信息无关。E. 音频转文本用于音频处理，与图片无关。

## ✉️ 评价反馈
感谢你学习阿里云大模型ACP认证课程，如果你觉得课程有哪里写得好、哪里写得不好，期待你[通过问卷提交评价和反馈](https://survey.aliyun.com/apps/zhiliao/Mo5O9vuie)。

你的批评和鼓励都是我们前进的动力。